<a href="https://www.kaggle.com/code/kennytanner/store-sales-forecasting-with-boosted-hybrid-model?scriptVersionId=120663817" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Please take a look at my Store Sales Forecasting EDA Notebook if you haven't yet at
https://www.kaggle.com/kennytanner/store-sales-forecasting-extensive-eda/

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_squared_log_error

from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import pickle

import warnings
from IPython import get_ipython
get_ipython().config.InlineBackend.figure_format = 'retina'
warnings.simplefilter("ignore")

# Contents
## 1. Big Decisions
#### 1.1. Model Picks and Sacrificing Data
#### 1.2. Imputting Zeros
## 2. Hybrid Class
## 3. Feature Engineering
#### 3.1. Dates
#### 3.2. X1 Features
#### 3.3. X2 Features
#### 3.4. Checking the Feature Set
## 4. Validation & Error Analysis
## 5. Predictions
## 6. Results

In [ ]:
df_sales= pd.read_csv(
    '/kaggle/input/store-sales-time-series-forecasting/train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True)

df_test = pd.read_csv(
    '/kaggle/input/store-sales-time-series-forecasting/test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True)

df_stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
df_transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv', parse_dates=['date'])
df_oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates=['date'])

df_holidays_events = pd.read_csv(
    "/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',},
    parse_dates=['date'],
    infer_datetime_format=True)

# 1. Big Decisions
## 1.1. Model Picks & Sacrificing Data
These required some thought and testing. So define here to avoid careless mistakes:

In [ ]:
# time range to train before train test split
full_train_start_day = datetime.datetime(2015, 6, 16) 
full_train_end_day = datetime.datetime(2017, 8, 15)

# time range for train validation split 
train_start_day = full_train_start_day
train_end_day = datetime.datetime(2017, 7, 30)
val_start_day = datetime.datetime(2017, 7, 31)
val_end_day = datetime.datetime(2017, 8, 15)

# time range of test set
test_start_day = datetime.datetime(2017, 8, 16)
test_end_day = datetime.datetime(2017, 8, 31)

The data pre June 2015 is very patchy and better results are found by removing it completely.

There was an Eartherquake in Ecuador over this period, killing nearly 700 people, and injuring 16,000. https://en.wikipedia.org/wiki/2016_Ecuador_earthquake

- It destroyed Manta's central shopping district. With store 53, open in Manta at the time, being impacted and store 52 opening locally in April 2017 also.
- This was only 170 km (110 mi) from the capital Quito, where it was also felt strongly and 18 of the stores are located.

**Please see my exploratory data analysis notebook for store locations and detailed EDA**
(https://www.kaggle.com/kennytanner/store-sales-forecasting-extensive-eda/)

Selecting the number of days for the second model to ignore! (max_lag)
- Introducing lags and rolling statistics will create NaNs and inacurate statistics. This occurs on the edges of the data as the information required to produce them falls outside the data available.

**Importantly:**
- The first model will be a feature-transforming algorithm. Its main purpose is to find any trend and extrapolate. (Target-transforming models cannot do this.)
- Then second model will be target-transforming and will be fit on the residuals of the first model. This model is intended to be good at capturing the interactions.
- By producing the lag features here we can just tell this model to ignore the effected rows, without loosing data at every forecasting step.

In [ ]:
max_lag = 7

### Forecasting Day by Day and then Refitting All Days
- To forecast 1 day at a time (so that lag features can be utilised)
- Then each new forecasted day is then factored in. Such that the other forecasted days are then reforecast using the larger data set

In [ ]:
mod_1 = LinearRegression()
mod_2 = XGBRegressor()

- I trialed a few different models. Lasso performed better than ridge regression. Perhaps some of the features were more noise than aid (lasso can remove their contribution by sending the coefficients to zero). I did orginally think ridge would perform better, especially when I could add in, and play around with feature engingeering with the second model, so I thought this was interesting.

- In the end LinearRegression performed the best. Linear regression does not impose any constraints on the model parameters. So can fit more complex models without any regularization. Lasso imposes a penalty on large coefficients, which can restrict the model's ability to fit complex models.

- With the trends removed, a nice and powerful gradient decent model is in order to fit to the residuals. And XGBoost performed well as the second model.

## 1.2. Imputting Zeros

In [ ]:
Not_sold = df_sales.groupby(["store_nbr", "family"]).sum().reset_index().sort_values(["family","store_nbr"])
Not_sold = Not_sold[Not_sold.sales == 0]
Not_sold.store_nbr = Not_sold.store_nbr.astype('str')
Not_sold

While I do appreciate that this is leakage into the validation set, aren't we trying to predict the test data not the validation data in the end.

If that is the goal, and we have no data but zeros to train on for these families at these stores. In my opinion, our best bet is to see if there is any indication that this will change over the period we are forecasting. And if not, then to predict zeros!

So here I check if there are any 'department opening' sales, to indicate that this is the case.

In [ ]:
Test_promos = df_test.groupby(["store_nbr", "family"]).sum().reset_index().sort_values(["family", "store_nbr"])
Test_promos = Test_promos[Test_promos.onpromotion>0]

Test_promos = Test_promos.set_index(["store_nbr", "family"]).index

Set_to_check = set(Not_sold.set_index(["store_nbr", "family"]).index)
Started_promotion_in_test_period = [x for x in Set_to_check if x in Test_promos]

Started_promotion_in_test_period

Looks like Lawn and Garden Departments have opened up in stores 14, 30 and 54, and we have an interesting situation at play!

Fortunately the model will still learn from all the other data we have and will predict some sales. Probably not the best predictions, and I would really love to hear if anyone knows some good tactics! Please post in the comments if so!

Perhaps identifying items that went from no sales ever, to sales, in a feature (called say "first_15_days_open") and identifying Lawn and Garden in stores 14, 30 & 54 as those over our test data date range. That would be interesting. I may still come back and do this (in which case you probably won't be reading this). But it would certainly be unwise to impute zeros here.

On the otherhand, those stores that have never sold books, baby care or ladieswear, surely they would at least have an opening sale on? If I spoke better Spanish this might be easy to research but for now I will impute zeros for those that have zero sales thus far and also no upcoming promotions.

In [ ]:
zeros_to_impute = list(Not_sold.set_index(["store_nbr", "family"]).index.drop(Started_promotion_in_test_period).values)
#zeros_to_impute

# 2. Hybrid Class

In [ ]:
class Hybrid():
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None
        self.stack_cols = None
        self.y_resid = None

    def fit1(self, X_1, y, stack_cols=None):
        self.model_1.fit(X_1, y) # train 1st model
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1), # predict from 1st model
            index=X_1.index,
            columns=y.columns,
        )
        self.y_resid = y - y_fit # residuals from 1st model
        self.y_resid = self.y_resid.stack(stack_cols).squeeze()  # wide to long
        
    def fit2(self, X_2, first_n_rows_to_ignore, stack_cols=None):
        self.model_2.fit(X_2.iloc[first_n_rows_to_ignore*1782: , :], self.y_resid.iloc[first_n_rows_to_ignore*1782:]) # Train 2nd model
        self.y_columns = y.columns # Save for predict method
        self.stack_cols = stack_cols # Save for predict method

    def predict(self, X_1, X_2, first_n_rows_to_ignore):
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1.iloc[first_n_rows_to_ignore: , :]), # predictions from 1st model
            index=X_1.iloc[first_n_rows_to_ignore: , :].index,
            columns=self.y_columns,
        )
        y_pred = y_pred.stack(self.stack_cols).squeeze()  # wide to long
        y_pred += self.model_2.predict(X_2.iloc[first_n_rows_to_ignore*1782: , :]) # Add model_2 predictions to model_1 predictions
        return y_pred.unstack(self.stack_cols)

# 3. Feature Engineering
The final selection of features shown here result from:
- EDA (https://www.kaggle.com/kennytanner/store-sales-forecasting-extensive-eda/)
- Validation experiments with combinaions that appeared promising
- Error analysis on the validation set
- Care not to overtrain to the test set (making selections based on validation - to relfect a real production and deployment scenario)

I classify stores open before the data as mature stores, as this matches the stores dataframe classification.

Distribution of sales over time, for stores are commonly seen to have features associated with being new in comparison to having "matured". (A spike in interest on opening and then often a lower sales rate than those who have developed a regular custmer base.)
Therefore it may help to differentiate those stores that have passed these initial stages from the newbies on the block, and including their opening dates.

In [ ]:
daily_sales_sum = (
    df_sales.drop(columns = 'onpromotion')
    .groupby(['date', 'store_nbr'])
    .sum()
    .reset_index())

mature_stores = daily_sales_sum[daily_sales_sum['date'] == '2013-01-02'].store_nbr.values

Also played around with using an opening date instead. I believe this might be better for newly opening stores but saw no improvement in validation and had no reason for concern in the 15 day test data period.

This might be something to look into futher though. Especially in implementation over the long term.

In [ ]:
stores_of_type = {}
for store_type in df_stores.type.unique():
    stores_of_type[store_type] = list(map(str, df_stores[df_stores.type == store_type].store_nbr))
    print(f'store type {store_type} = {stores_of_type[store_type]}')

In [ ]:
def add_store_types(df=df_sales):
    for store_type in df_stores.type.unique():
        df[f'store_type_{store_type}'] = df.store_nbr.isin(stores_of_type[store_type])
    return df

In [ ]:
df_sales['date'] = df_sales.date.dt.to_period('D')

# And to get the missing Christmas days, in the multi index form
multiindex = pd.MultiIndex.from_product([df_sales["store_nbr"].unique(),
                                      df_sales["family"].unique(),
                                      pd.date_range(start="2013-1-1", end="2017-8-15", freq="D").to_period('D')]
                                     ,names=["store_nbr","family", "date"])
df_sales= df_sales.set_index(["store_nbr","family", "date"]).reindex(multiindex, fill_value=0).sort_index()


df_sales= df_sales.unstack(['store_nbr', 'family'])
df_sales= df_sales.stack(['store_nbr', 'family'])

df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

In [ ]:
df_oil['date'] = df_oil.date.dt.to_period('D')
df_oil = df_oil.set_index('date')

# Linear Interpolation
df_oil = df_oil.dcoilwtico.resample('D').sum().reset_index()

df_oil['dcoilwtico'] = np.where(df_oil['dcoilwtico'] == 0, np.nan, df_oil['dcoilwtico'])
df_oil['dcoilwtico_interpolated'] = df_oil.dcoilwtico.interpolate()

df_oil = df_oil.set_index('date')
df_oil.dcoilwtico_interpolated = df_oil.dcoilwtico_interpolated.fillna(method='bfill')

In [ ]:
def threshold_for_expensive(threshold=60):
    df_oil["expensive_oil"]=df_oil.dcoilwtico_interpolated >= threshold
    return df_oil.drop(columns=['dcoilwtico', 'dcoilwtico_interpolated'])

In [ ]:
def expensive_oil_add(df, threshold=60):
    df = df.join(threshold_for_expensive(threshold))
    return df

## 3.1. Dates
Correcting the Good Friday error immediately

In [ ]:
df_holidays_events['date'] = df_holidays_events['date'].replace({'2013-04-29':pd.to_datetime('2013-03-29')})
# Quite an easy data collection error to check and correct - Good Friday was March 29th in 2013 (in Columbia too)
df_holidays_events = df_holidays_events.set_index('date').to_period('D').sort_index()

Without the Fourier features (seasonal oscillations), one could simply remove the missing Christmas days....

In [ ]:
# missing_Xmas_days = list((pd.date_range(start="2013-01-01", end="2017-08-15").difference(df_sales.date)))

# for _, day in enumerate(missing_Xmas_days):
#     missing_Xmas_days[_] = day.to_period(freq='D')

# missing_Xmas_days

# calendar = calendar.drop(missing_Xmas_days, axis=0)

# df_sales = df_sales.set_index(["date", "store_nbr", "family"])
# df_sales

While there are also smaller holidays, the national holidays effect all stores and are by far the largest proportion to effect any store, by a massive margin so lets start by identifying the dates and effects of those.

In [ ]:
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31')).to_period('D')
calendar['dofw'] = calendar.index.dayofweek

calendar['wd'] = True
calendar.loc[calendar.dofw > 4, 'wd'] = False

In [ ]:
National_Holidays = df_holidays_events[df_holidays_events.locale == 'National'].copy()

# Having more than one holiday on the same day messes with the shape of the datafrae by adding row
# The model should do perfectly fine with only one holiday identified for the date (so just using the first)
National_Holidays = National_Holidays.groupby(National_Holidays.index).first()

calendar = calendar.merge(National_Holidays, how='left', left_index=True, right_index=True)

calendar.loc[calendar.type == 'Bridge'  , 'wd'] = False
calendar.loc[calendar.type == 'Work Day', 'wd'] = True
calendar.loc[calendar.type == 'Transfer', 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'wd'] = False
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'wd'] = True

calendar

In [ ]:
df_wd = calendar
df_wd["date"] = df_wd.index
df_wd = df_wd.set_index("date")
df_wd = df_wd.drop(columns=['dofw', 'type', 'locale', 'locale_name', 'description', 'transferred'])

def wd_add(df, start_date, end_date):  # for long format
    df = df.join(df_wd.loc[start_date:end_date])
    return df

## 3.2. X1 Features

In [ ]:
def make_DeterministicProcess_features(df):
    y = df.loc[:, 'sales']
    fourier_m = CalendarFourier(freq='M', order=4) # monthly showed good results
    dp = DeterministicProcess(                     # weekly and annual performed badly on validation set
        index=y.index,
        constant=True,
        order=1,
        seasonal=True,
        additional_terms=[fourier_m],
        drop=True,
    )
    return y, dp

In [ ]:
def make_X1_features(df, start_date, end_date, is_test_set=False):
    if is_test_set:
        X1 = df.rename_axis('date')
    else:
        y, dp = make_DeterministicProcess_features(df)
        X1 = dp.in_sample()

    # All closed Chirstmas days so far.
    # And all the stores were closed on new year except 25 and 36 sometimes
    # But in order to make the fourier features it was important to include these days
    # Therefore it is important to identify them!
    
    X1['NewYear'] = (X1.index.dayofyear == 1)   
    X1['Christmas'] = (X1.index=='2016-12-25') | (X1.index=='2015-12-25') | (X1.index=='2014-12-25') | (X1.index=='2013-12-25')
    
    X1['wd']   = calendar.loc[start_date:end_date]['wd'].values    
    X1['type'] = calendar.loc[start_date:end_date]['type'].values
    X1['description'] = calendar.loc[start_date:end_date]['description'].values
    X1 = pd.get_dummies(X1, columns=['type', 'description'], drop_first=False)
    X1.drop(['type_Work Day', 'type_Event', 'type_Holiday'], axis=1, inplace=True)
    
    if is_test_set:
        return X1
    else:
        return X1, y, dp

## 3.3. X2 Features

I classify stores open before the data as mature stores, as this matches the stores dataframe classification.

Distribution of sales over time, for stores are commonly seen to have features associated with being new in comparison to having "matured". (A spike in interest on opening and then often a lower sales rate than those who have developed a regular custmer base.)
Therefore it may help to differentiate those stores that have passed these initial stages from the newbies on the block, and including their opening dates.

In [ ]:
def encode_categoricals(df, columns):
    le = LabelEncoder()
    for col in columns:
        df[col] = le.fit_transform(df[col])
    return df

In [ ]:
def make_X2_lags(ts, lags, lead_time=1, name='y', stack_cols=None):
    ts = ts.unstack(stack_cols)
    df = pd.concat(
        {
            f'{name}_lag_{i}': ts.shift(i, freq="D")
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)
    df = df.stack(stack_cols).reset_index()
    df = encode_categoricals(df, stack_cols)
    df = df.set_index('date').sort_values(by=stack_cols)
    return df

In [ ]:
def make_X2_features(df, y_resid, start_date, end_date):
    stack_columns = ['store_nbr', 'family']
    
    # promo_lag features
    promo_target_df = make_X2_lags(df['onpromotion'].squeeze(), lags=max_lag, name='promo', stack_cols=['store_nbr', 'family'])
    
    promo_target_df['promo_mean_rolling_7'] = promo_target_df['promo_lag_1'].rolling(window=7, center=False).mean()
    promo_target_df['promo_median_rolling_70'] = promo_target_df['promo_lag_1'].rolling(window=70, center=False).median().fillna(method='bfill')
    promo_target_df['promo_median_rolling_140'] = promo_target_df['promo_lag_1'].rolling(window=140, center=False).median().fillna(method='bfill')
    # for larger rolling window medians, backfilling seems reasonable as medians shouldn't change too much.
    
    # y_lag features
    sales_target_df = make_X2_lags(y_resid, lags=max_lag, name='y_res', stack_cols=stack_columns)
    
    sales_target_df['y_mean_rolling_7'] = sales_target_df['y_res_lag_1'].rolling(window=7, center=False).mean()
    sales_target_df['y_median_rolling_14'] = sales_target_df['y_res_lag_1'].rolling(window=14, center=False).median().fillna(method='bfill')
    sales_target_df['y_median_rolling_70'] = sales_target_df['y_res_lag_1'].rolling(window=70, center=False).median().fillna(method='bfill')
    sales_target_df['y_median_rolling_140'] = sales_target_df['y_res_lag_1'].rolling(window=140, center=False).median().fillna(method='bfill')
    
    sales_target_df['y_last_half_year_max']= sales_target_df['y_res_lag_1'].rolling(window=182, center=False).max().fillna(method='bfill')
    sales_target_df['y_last_half_year_min']= sales_target_df['y_res_lag_1'].rolling(window=182, center=False).min().fillna(method='bfill')
    sales_target_df['y_last_month_std']= sales_target_df['y_res_lag_1'].rolling(window=29, center=False).std().fillna(method='bfill')
    
    # interactions and other features
    sales_target_df['lag1-2_week_median_sales'] = sales_target_df['y_res_lag_1'] - sales_target_df['y_median_rolling_14']
    sales_target_df = sales_target_df.drop(columns='y_median_rolling_14')
    
    df = df.reset_index(stack_columns)
    X2 = encode_categoricals(df, stack_columns)
    
    X2['NewYear'] = (X2.index.dayofyear == 1)
    X2['Christmas'] = (X2.index=='2016-12-25') | (X2.index=='2015-12-25') | (X2.index=='2014-12-25') | (X2.index=='2013-12-25')
    
    # All public sector wages in Ecuador are paid on the 15th and the last day of the month
    X2['wage_day'] = (X2.index.day == X2.index.daysinmonth) | (X2.index.day == 15)
    X2['wage_day_lag_1'] = (X2.index.day == 1) | (X2.index.day == 16)
    #X2['wage_day_lag_2'] = (X2.index.day == 2) | (X2.index.day == 17) # another surprise that even just a second lag was too many!
    
    X2['store_mature'] = X2['store_nbr'].isin(list(map(str, mature_stores)))
    
    X2 = expensive_oil_add(X2, threshold=60)
    X2 = add_store_types(X2)
    X2 = wd_add(X2, start_date, end_date)
    
    X2['promo_mean'] = X2.groupby(['store_nbr', 'family'])['onpromotion'].transform("mean") + 0.000001
    X2['promo_ratio'] = X2.onpromotion / (X2.groupby(['store_nbr', 'family'])['onpromotion'].transform("mean") + 0.000001)
    
    X2 = X2.merge(sales_target_df, on=['date', 'store_nbr', 'family'], how='left')
    X2 = X2.merge(promo_target_df, on=['date', 'store_nbr', 'family'], how='left')
    return X2

## 3.4. Checking the Feature Set
This is a simple and much faster look into the outcomes. More for debugging than for experimentation (no one day at a time and refitting). The validation and error analysis (section 4) provide better feed back on how well the features perform.

In [ ]:
%%time
df_sales_in_date_range = df_sales.unstack(['store_nbr', 'family']).loc[full_train_start_day:full_train_end_day]

model = Hybrid(model_1=mod_1, model_2=mod_2)

X_1, y, dp = make_X1_features(df_sales_in_date_range, full_train_start_day, full_train_end_day) 
model.fit1(X_1, y, stack_cols=['store_nbr', 'family'])
X_2 = make_X2_features(df_sales_in_date_range
                       .drop('sales', axis=1)
                       .stack(['store_nbr', 'family']),
                       model.y_resid, full_train_start_day, full_train_end_day)
model.fit2(X_2, max_lag, stack_cols=['store_nbr', 'family'])

y_pred = model.predict(X_1, X_2, max_lag).clip(0.0)

In [ ]:
def truncateFloat(data):
    return tuple( ["{0:.2f}".format(x) if isinstance(x,float) else (x if not isinstance(x,tuple) else truncateFloat(x)) for x in data])

temp = X_2[(X_2.store_nbr == 1) & (X_2.family == 3)]
temp.iloc[max_lag: , :].apply(lambda s: truncateFloat(s))

temp.apply(lambda s: truncateFloat(s)).head(10) # fit method will skip over the nan rows

In [ ]:
X_1.iloc[max_lag: , :].apply(lambda s: truncateFloat(s))

A look at how the model fits the training data

In [ ]:
#display(df_sales.index.get_level_values('family').unique())
STORE_NBR = '1'  # 1-54
FAMILY = 'BEVERAGES' 

ax = y.loc(axis=1)[STORE_NBR, FAMILY].plot(figsize=(16, 4), color='silver')
ax = y_pred.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax,  color='cornflowerblue')
plt.xlabel("Sales")
ax.set_title(f'{FAMILY.title()} Sales at Store {STORE_NBR}')

# 4. Validation & Some Error Analysis

In [ ]:
training_days = (train_end_day - train_start_day).days + 1
validation_days = (val_end_day - val_start_day).days + 1
print("training data set (excluding validation days) has", training_days, "days")
print("validation data set has", validation_days, "days\n")
print("validating a hybrid with the first model being: \n \n ", mod_1)
print('\n and the second being: \n \n ', mod_2)
print('\n \n with max_lag=', max_lag, 'days')

df_sales_in_date_range = df_sales.unstack(['store_nbr', 'family']).loc[train_start_day:train_end_day]
sales_in_val_range = df_sales.unstack(['store_nbr', 'family']).loc[val_start_day:val_end_day]
y_val = y[val_start_day:val_end_day]

model_for_val = Hybrid(model_1=mod_1, model_2=mod_2)

In [ ]:
%%time
#initial fit on train portion of train/val split
X_1_train, y_train, dp_val = make_X1_features(df_sales_in_date_range, train_start_day, train_end_day) 
model_for_val.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family'])
X_2_train = make_X2_features(df_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_val.y_resid, full_train_start_day, full_train_end_day)
model_for_val.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])
y_fit = model_for_val.predict(X_1_train, X_2_train, max_lag).clip(0.0)

# loop through forecast, one day at a time
dp_for_full_X1_val_date_range = dp_val.out_of_sample(steps=validation_days)
for step in range(validation_days):
        dp_steps_so_far = dp_for_full_X1_val_date_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_val.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([df_sales_in_date_range,
                                       sales_in_val_range.loc[val_start_day:val_start_day+pd.Timedelta(days=step), :]])
        X_1_val = make_X1_features(X_1_combined_dp_data, train_start_day, val_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_val = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_val.y_resid, train_start_day, val_start_day+pd.Timedelta(days=step))

        y_pred_combined = model_for_val.predict(X_1_val, X_2_val, max_lag).clip(0.0)

        y_plus_y_val = pd.concat([y_train, y_pred_combined.iloc[-(step+1):]]) 
        model_for_val.fit1(X_1_val, y_plus_y_val, stack_cols=['store_nbr', 'family']) 
        model_for_val.fit2(X_2_val, max_lag, stack_cols=['store_nbr', 'family'])

        rmsle_valid = mean_squared_log_error(y_val.iloc[step:step+1], y_pred_combined.iloc[-1:]) ** 0.5
        print(f'Validation RMSLE: {rmsle_valid:.5f}', "for", val_start_day+pd.Timedelta(days=step)) # To see progress

In [ ]:
y_pred = y_pred_combined[val_start_day:val_end_day]

for zeros in zeros_to_impute:
    y_pred[zeros]=0.0

print("\ny_pred: ")
display(y_pred.apply(lambda s: truncateFloat(s)))
    
if type(model_for_val.model_2) == XGBRegressor:
        pickle.dump(model_for_val.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

RMSLE is calculated as:
$$\sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2}$$
where:


 n is the total number of instances,

$\hat{y}_i$ is the predicted value of the target for instance (i),

$y_i$ is the actual value of the target for instance (i), and,

log is the natural logarithm.

In [ ]:
rmsle_train = mean_squared_log_error(y_train.iloc[max_lag: , :].clip(0.0), y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val.clip(0.0), y_pred) ** 0.5
print()
print(f'Training RMSLE: {rmsle_train:.5f}')
print(f'Validation RMSLE: {rmsle_valid:.5f}')
    
y_predict = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y_val.stack(['store_nbr', 'family']).reset_index().copy()
y_target.rename(columns={y_target.columns[3]:'sales'}, inplace=True)
y_target['sales_pred'] = y_predict[0].clip(0.0) # Sales should be >= 0
y_target['store_nbr'] = y_target['store_nbr'].astype(int)

print('\nValidation RMSLE by family')
display(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).sort_values(ascending=False))

print('\nValidation RMSLE by store')
display(y_target.sort_values(by="store_nbr").groupby('store_nbr').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])).sort_values(ascending=False))

In [ ]:
y_target

In [ ]:
def error_analysis(FAMILY=None, STORE_NBR= None):
    y=y_target.set_index('date')
    if FAMILY:
        if FAMILY in y_target.family.unique():
            y=y[y.family == FAMILY]
            if not STORE_NBR:
                y.groupby('date').sales.mean().plot(color='silver', label='mean sales')
                y.groupby('date').sales_pred.mean().plot(color='cornflowerblue', label='mean prediction')
                plt.legend()
                plt.ylabel('Mean Sales')
                plt.xlabel('Date')
                plt.title(f'All Stores Comparison for {FAMILY.title()}')
                plt.show()
                for STORE_NBR in range(1,55):
                    plt.clf()
                    y.sales[y.store_nbr == STORE_NBR].plot(color='black', label='sales')
                    y.sales_pred[y.store_nbr == STORE_NBR].plot(color='fuchsia', label='prediction')
                    plt.legend()
                    plt.ylabel('Sales')
                    plt.xlabel('Date')
                    plt.title(f'Store #{STORE_NBR} {FAMILY.title()}')
                    plt.show()
                return
        else:
            return (f"departments (families) available are {y_target.family.unique()}")
    if STORE_NBR:
        if STORE_NBR <= 54:
            if FAMILY:
                y.sales[y.store_nbr == STORE_NBR].plot(color='black', label='sales')
                y.sales_pred[y.store_nbr == STORE_NBR].plot(color='fuchsia', label='prediction')
                plt.ylabel('Sales')
                plt.xlabel('Date')
                plt.legend()
                plt.title(f"Store {STORE_NBR} {FAMILY.title()} Comparison")
                plt.show()
            else:
                y=y[y.store_nbr == STORE_NBR]
                y.groupby('date').sales.mean().plot(color='silver', label='mean sales')
                y.groupby('date').sales_pred.mean().plot(color='tomato', label='mean prediction')
                plt.legend()
                plt.ylabel('Mean Sales')
                plt.xlabel('Date')
                plt.title(f'All Departments (Families) Comparison for Store #{STORE_NBR}')
                plt.show()
                for FAMILY in y_target.family.unique():
                    plt.clf()
                    y.sales[y.family == FAMILY].plot(color='black', label='sales')
                    y.sales_pred[y.family == FAMILY].plot(color='fuchsia', label='prediction')
                    plt.ylabel('Sales')
                    plt.xlabel('Date')
                    plt.legend()
                    plt.title(f'Store #{STORE_NBR} {FAMILY.title()}')
                    plt.show()
        else:
            return "max number of stores=54"
    else:
        for STORE_NBR in range(1,55):
            y1=y[y.store_nbr == STORE_NBR].groupby(['date']).mean()
            y1.groupby('date').sales.mean().plot(color='silver', label='mean sales')
            y1.groupby('date').sales_pred.mean().plot(color='cornflowerblue', label='mean prediction')
            plt.legend()
            plt.ylabel('Mean Sales')
            plt.xlabel('Date')
            plt.title(f'Average Sales Across all Departments (Families) for Store # {STORE_NBR}')
            plt.show()
            plt.clf()
        for FAMILY in y_target.family.unique():
            y1=y[y.family == FAMILY].groupby(['date']).mean()
            y1.groupby('date').sales.mean().plot(color='silver', label='mean sales')
            y1.groupby('date').sales_pred.mean().plot(color='tomato', label='mean prediction')
            plt.legend()
            plt.ylabel('Mean Sales')
            plt.xlabel('Date')
            plt.title(f'Average Sales Across all Stores for {FAMILY.title()}')
            plt.show()
            plt.clf()

In [ ]:
error_analysis(FAMILY="SCHOOL AND OFFICE SUPPLIES")

To capture the back to school season perhaps one hot encoding the month of the year (or even week of the year if you aren't overly concerned with massively increasing the dimentionality) would be a significant help. Label encoding isn't really appropriate as yearly seasons like this are unlikely to be linear! I did play around with label enoding over the week and month thinking their might be some relationships in there after wage day or weekend excitement. But took them out, finding the working day and wage day related features to be better.

My EDA (https://www.kaggle.com/kennytanner/store-sales-forecasting-extensive-eda/) did not show significant seasonality in the School and Office Supplies anyway, and I expect the first model captured most already, so I didn't worry about it in the end. 

But if you did go ahead and try that (or have other interesteing suggestions) that please let me know in the comments!

# 5. Predictions

In [ ]:
train_days = (full_train_end_day - full_train_start_day).days + 1
test_days = (test_end_day - test_start_day).days + 1

print("data trained over", train_days, "days")
print("test forecasting period is", test_days, "days through", test_end_day, "\n")
print("validating a hybrid with the first model being: \n \n ", mod_1)
print('\n and the second being: \n \n ', mod_2)
print('\n \n with max_lag=', max_lag, ' days')
df_sales_in_date_range = df_sales.unstack(['store_nbr', 'family']).loc[full_train_start_day:full_train_end_day]
sales_in_test = df_test.unstack(['store_nbr', 'family']).drop('id', axis=1)

model_for_test = Hybrid(model_1=mod_1, model_2=mod_2)

In [ ]:
%%time
#initial fit on train portion of train/test split
X_1_train, y_train, dp_test = make_X1_features(df_sales_in_date_range, full_train_start_day, full_train_end_day)
model_for_test.fit1(X_1_train, y_train, stack_cols=['store_nbr', 'family']) 
X_2_train = make_X2_features(df_sales_in_date_range
                           .drop('sales', axis=1)
                           .stack(['store_nbr', 'family']),
                           model_for_test.y_resid, full_train_start_day, full_train_end_day)
model_for_test.fit2(X_2_train, max_lag, stack_cols=['store_nbr', 'family'])

dp_for_full_X1_test_date_range = dp_test.out_of_sample(steps=test_days)
for step in range(test_days):
        dp_steps_so_far = dp_for_full_X1_test_date_range.loc[test_start_day:test_start_day+pd.Timedelta(days=step),:]

        X_1_combined_dp_data = pd.concat([dp_test.in_sample(), dp_steps_so_far])
        X_2_combined_data = pd.concat([df_sales_in_date_range,
                                       sales_in_test.loc[test_start_day:test_start_day+pd.Timedelta(days=step), :]])
        X_1_test = make_X1_features(X_1_combined_dp_data, train_start_day, test_start_day+pd.Timedelta(days=step), is_test_set=True)
        X_2_test = make_X2_features(X_2_combined_data
                                    .drop('sales', axis=1)
                                    .stack(['store_nbr', 'family']),
                                    model_for_test.y_resid, train_start_day, test_start_day+pd.Timedelta(days=step))

        y_forecast_combined = model_for_test.predict(X_1_test, X_2_test, max_lag).clip(0.0)

        y_plus_y_test = pd.concat([y_train, y_forecast_combined.iloc[-(step+1):]])
        model_for_test.fit1(X_1_test, y_plus_y_test, stack_cols=['store_nbr', 'family'])
        model_for_test.fit2(X_2_test, max_lag, stack_cols=['store_nbr', 'family'])
        print("finished forecast for", test_start_day+pd.Timedelta(days=step)) # To see progress
        
display(y_forecast_combined[test_start_day:test_end_day])

In [ ]:
y_forecast = pd.DataFrame(y_forecast_combined.loc[test_start_day:test_end_day].clip(0.0))
    
if type(model_for_test.model_2) == XGBRegressor:
        pickle.dump(model_for_test.model_2, open("xgb_temp.pkl", "wb"))
        m2 = pickle.load(open("xgb_temp.pkl", "rb"))
        print("XGBRegressor paramaters:\n",m2.get_xgb_params(), "\n")

In [ ]:
for zeros in zeros_to_impute:
    y_forecast[zeros]=0.0
    y_fit[zeros]=0.0

# 6. Results

In [ ]:
# display(df_sales.index.get_level_values('family').unique())
STORE_NBR = '1'  # 1 - 54
FAMILY = 'PRODUCE'

ax = y.loc(axis=1)[STORE_NBR, FAMILY].plot(figsize=(16, 4), color='dimgrey')
ax = y_pred.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax, color='greenyellow', marker='.')
ax = y_forecast.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax, color='blueviolet', marker='.')
ax.set_title(f'{FAMILY.title()} Sales at Store {STORE_NBR}')
ax.set_ylabel(f'{FAMILY.title()} Sales')

# fit = black
# validation prediction (before model was retrained for final predictions) = green
# forecast = purple

In [ ]:
NUM_FAMILIES = 33   # 1 - 33
STORE_NBR = '1'    # 1 - 54

y_for_store = y.loc(axis=1)[STORE_NBR]
y_fit_for_store = y_fit.loc(axis=1)[STORE_NBR]
y_pred_for_store = y_pred.loc(axis=1)[STORE_NBR]
y_forecast_for_store = y_forecast.loc(axis=1)[STORE_NBR]
families = y_for_store.columns[0:NUM_FAMILIES]

axs = y_for_store.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(16, 3*NUM_FAMILIES), alpha=0.6,
)
_ = y_fit_for_store.loc(axis=1)[families].plot(subplots=True, sharex=True, color='dimgrey', ax=axs)
_ = y_pred_for_store.loc(axis=1)[families].plot(subplots=True, sharex=True, color='greenyellow', ax=axs, marker='.')
_ = y_forecast_for_store.loc(axis=1)[families].plot(subplots=True, sharex=True, color='slateblue', ax=axs, marker='.')

for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(f'{family.title()} Sales')
    ax.set_title(f'{family.title()} Sales at Store {STORE_NBR}')
    ax.tick_params(labelbottom=True)

plt.tight_layout()
plt.show()
    
# fit = black
# validation prediction (before model was retrained for final predictions) = green
# forecast = purple

In [ ]:
submission = y_forecast.stack(['store_nbr', 'family'])
submission = pd.DataFrame(submission, columns=['sales'])
submission = submission.join(df_test.id).reindex(columns=['id', 'sales'])
submission.to_csv('submission.csv', index=False)
submission

As always, thank you to the community. This has been a fun learning experiance. Please see below my main sources. They have been a massive help to me and I would recommend these to everyone in their own analysis journey with this dataset.

Hoang Pham Viet: Past Time Series competitions _ Winning solutions (https://www.kaggle.com/competitions/store-sales-time-series-forecasting/discussion/278372)

Ryan Holbrook: Kaggle Official Time Series Course (https://www.kaggle.com/learn/time-series)

FilterJoe: Time Series Bonus Lesson (Unofficial) (https://www.kaggle.com/code/filterjoe/time-series-bonus-lesson-unofficial/notebook)

Ekrem Bayar: Store Sales TS Forecasting - A Comprehensive Guide (https://www.kaggle.com/code/ekrembayar/store-sales-ts-forecasting-a-comprehensive-guide/notebook)

Javier Ruiz: Oil Prices Matter More Than We Think(https://www.kaggle.com/competitions/store-sales-time-series-forecasting/discussion/298626)

Pratik Narkhede: What is onpromotion, type, cluster?(https://www.kaggle.com/competitions/store-sales-time-series-forecasting/discussion/289764)

The Devastator: Store Sales - Top Discussion Posts (https://www.kaggle.com/competitions/store-sales-time-series-forecasting/discussion/332439)

And just in case you missed the other links ;) My exploratory data analysis notebook is https://www.kaggle.com/kennytanner/store-sales-forecasting-extensive-eda/